In [ ]:
import pandas as pd
from embeds import multi_inner_align, standardize
from transformers import AutoTokenizer, AutoModel
import torch

sys.path.append('..')
from rca.rca import run_rca

In [ ]:
to_pull

## Extracting representations

In [ ]:
torch.random.manual_seed(42)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('CUDA is available. Using GPU.')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available. Using Apple's Metal.")
else:
    device = torch.device("cpu")
    print("No GPU or MPS available. Using CPU.")


model_name = ''
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

embeds



In [ ]:
# Comparing
to_compare = {}

# Aligning for fair comparison
to_compare = dict(zip(to_compare.keys(), multi_inner_align(to_compare.values())))

# Standardizing
to_compare = {name: standardize(embed) for name, embed in to_compare.items()}

# Loading norm data
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, low_memory=False, compression='zip')
norms_meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col='norm')
norms

In [ ]:
results = run_rca(to_compare, norms, norms_meta, n_jobs=10)
results

In [ ]:
# Adding norm category
results['norm_category'] = (
    results['norm']
    .apply(lambda norm: norms_meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

results_avg = (
    results[['norm_category', 'embed', 'r2_mean']]
    .groupby(['norm_category', 'embed'], as_index=False).median()
    .dropna()
)

results_avg_piv = results_avg.pivot(columns='embed', index='norm_category', values='r2_mean')
results_avg_piv.round(2)

In [ ]:
# Finding the top-performing fmri_text_denoise
sorted_overall = results_avg_piv.mean().sort_values(ascending=False)
sorted_overall

## Saving

In [ ]:
to_pull = set(
    pd.read_csv('../../data/psychNorms/psychNorms.zip', index_col=0, low_memory=False, compression='zip').index
)

